In [2]:
pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.4/240.4 KB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 KB 

In [7]:
pip install qiskit_machine_learning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 KB 13.5 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [3]:
import qiskit
from qiskit import quantum_info, QuantumCircuit
from qiskit.execute_function import execute
from qiskit import BasicAer
import numpy as np
import pickle
import json
import os
import sys
from collections import Counter
from sklearn.metrics import mean_squared_error
from typing import Dict, List
from sklearn.decomposition import PCA
import tqdm


import matplotlib.pyplot as plt

if len(sys.argv) > 1:
    data_path = sys.argv[1]
else:
    data_path = '.'

#define utility functions

def simulate(circuit: qiskit.QuantumCircuit) -> dict:
    """Simulate the circuit, give the state vector as the result."""
    backend = BasicAer.get_backend('statevector_simulator')
    job = execute(circuit, backend)
    result = job.result()
    state_vector = result.get_statevector()
    
    histogram = dict()
    for i in range(len(state_vector)):
        population = abs(state_vector[i]) ** 2
        if population > 1e-9:
            histogram[i] = population
    
    return histogram


def histogram_to_category(histogram):
    """This function takes a histogram representation of circuit execution results, and processes into labels as described in
    the problem description."""
    assert abs(sum(histogram.values())-1)<1e-8
    positive=0
    for key in histogram.keys():
        digits = bin(int(key))[2:].zfill(20)
        if digits[-1]=='0':
            positive+=histogram[key]
        
    return positive

def count_gates(circuit: qiskit.QuantumCircuit) -> Dict[int, int]:
    """Returns the number of gate operations with each number of qubits."""
    counter = Counter([len(gate[1]) for gate in circuit.data])
    #feel free to comment out the following two lines. But make sure you don't have k-qubit gates in your circuit
    #for k>2
    for i in range(3,20):
        assert counter[i]==0
        
    return counter


def image_mse(image1,image2):
    # Using sklearns mean squared error:
    # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html
    return mean_squared_error(255*image1,255*image2)

# [normalize]
def normalize(row):
    #We calculate the squareroot of the sum of the square values of the row
    normalization_factor = np.sqrt(np.sum(row**2)) 
    if normalization_factor == 0.:
        #If the sum is zero we return a 0
        return 0.0
    #Else we divide each value between the sum value above
    row = row / normalization_factor
    return row, normalization_factor
#
data_path = "./data"

def test():
    #load the actual hackthon data (fashion-mnist)
    images=np.load(data_path+'/images.npy')
    labels=np.load(data_path+'/labels.npy')
    
    #test part 1
    global pca, AMPLITUDE_ENCODING_N_QUBITS,IMAGE_SIZE, dataset

    AMPLITUDE_ENCODING_N_QUBITS = 4
    IMAGE_SIZE = 28
    N_IMAGES = len(images)

    pca = PCA(n_components=2**AMPLITUDE_ENCODING_N_QUBITS)
    
    data = images.reshape(N_IMAGES, IMAGE_SIZE * IMAGE_SIZE)
    pca.fit(data)
    
    # Apply dimensionality reduction on your data
    images_pca = pca.transform(data)
    
    dataset = []
    for i in tqdm.tqdm(range(len(images))):
        image_pca = images_pca[i]
        image_pca_min_ = image_pca.min()
        image_pca_positive = image_pca - image_pca_min_
        
        dataset_i = {}
        
        normalized_state, normalization_factor = normalize(image_pca_positive)
        
        dataset_i["image"] = images[i]
        dataset_i["image_vector"] = normalized_state
        dataset_i["label"] = labels[i]
        dataset_i["normalization_factor"] = normalization_factor
        dataset_i["pca_min_"] = image_pca_min_
        
        dataset.append(dataset_i)
    
    n=len(images)
    mse=0
    gatecount=0

    for i in tqdm.tqdm(range(n)):
        #encode image into circuit
        circuit,image_re=run_part1(dataset[i]["image_vector"])
        image_re = np.asarray(image_re)

        #count the number of 2qubit gates used
        gatecount += count_gates(circuit)[2]

        #calculate mse
        mse+=image_mse(dataset[i]["image"], image_re)

    #fidelity of reconstruction
    f=1-mse/n
    gatecount=gatecount/n

    #score for part1
    score_part1=f*(0.999**gatecount)
    
    print(score_part_1)

    #test part 2
    
    score=0
    gatecount=0
    n=len(images)

    for i in range(n):
        #run part 2
        circuit,label=run_part2(images[i])

        #count the gate used in the circuit for score calculation
        gatecount+=count_gates(circuit)[2]

        #check label
        if label==labels[i]:
            score+=1
    
    #score
    score=score/n
    gatecount=gatecount/n

    score_part2=score*(0.999**gatecount)
    
    print(score_part1, ",", score_part2, ",", data_path, sep="")


############################
#      YOUR CODE HERE      #
############################
def encode(image):
    qc = QuantumCircuit(AMPLITUDE_ENCODING_N_QUBITS)

    qc.initialize(image)
    
    for i in range(AMPLITUDE_ENCODING_N_QUBITS + 2):
        qc = qc.decompose()
        
    return qc

def decode(histogram):
    histogram_array = np.zeros(2 ** AMPLITUDE_ENCODING_N_QUBITS)
    for i in range(2 ** AMPLITUDE_ENCODING_N_QUBITS):
        histogram_array[i] = histogram.get(i, 0)
        
    root = np.sqrt(histogram_array)
    
    root = root * dataset[i]["normalization_factor"]
    
    root = root + dataset[i]["pca_min_"]

    reconstruction = pca.inverse_transform([root])

    
    image = reconstruction.reshape(IMAGE_SIZE, IMAGE_SIZE)
    
    return image

def run_part1(image):
    #encode image into a circuit
    circuit=encode(image)

    #simulate circuit
    histogram=simulate(circuit)

    #reconstruct the image
    image_re=decode(histogram)

    return circuit,image_re

def run_part2(image):
    # load the quantum classifier circuit
    classifier=qiskit.QuantumCircuit.from_qasm_file('quantum_classifier.qasm')
    
    #encode image into circuit
    circuit=encode(image)
    
    #append with classifier circuit
    nq1 = circuit.width()
    nq2 = classifier.width()
    nq = max(nq1, nq2)
    qc = qiskit.QuantumCircuit(nq)
    qc.append(circuit.to_instruction(), list(range(nq1)))
    qc.append(classifier.to_instruction(), list(range(nq2)))
    
    #simulate circuit
    histogram=simulate(qc)
        
    #convert histogram to category
    label=histogram_to_category(histogram)
    
    #thresholding the label, any way you want
    if label>0.5:
        label=1
    else:
        label=0
        
    return circuit,label

############################
#      END YOUR CODE       #
############################

test()

  0%|          | 8/2000 [00:01<07:06,  4.67it/s]


KeyboardInterrupt: ignored

In [18]:
from sklearn.utils import shuffle
from sklearn.metrics import log_loss
import tqdm
from qiskit.circuit.library import RealAmplitudes
from qiskit.utils import algorithm_globals


from scipy.optimize import minimize 


def ansatz(num_qubits):
    return RealAmplitudes(num_qubits, reps=5)

#Function to identify a function cost
'''
Input: An array of parameters(vector of complex amplitudes)
Output: Function cost
'''

def objective_function(params, images, labels, shots=1024, mini_batch=5, n_qubits=8):
    running_loss = 0
    images, labels = shuffle(images, labels)
    y_pred = []

    #For each item of the trainig set
    for i in tqdm.tqdm(range(mini_batch)):
        
        image = images[i]
        
        circuit = encode(image) #Create a inicial circuit
        vqc = ansatz(n_qubits)

        vqc = vqc.assign_parameters(params)

        #Create a new circuit based on the size of the initial circuit and the desired qubits to reduce
        circuit_full = QuantumCircuit(n_qubits)
    
        qubits = list(range(n_qubits))
        
        circuit_full.append(circuit.to_instruction(), qubits)
        circuit_full.append(vqc.to_instruction(), qubits)

        #Combine the initial circuit, the quantum variatinal circuit and the swap test
        #For the initial circuit and QVC we start at the qubit size_reduce + 1
        
        #Execute the circuit in the qasm_simulator
        histogram = simulate(circuit_full)

        pred_label = histogram_to_category(histogram)
        
        y_pred.append(pred_label)
    
    print(labels.shape)
    
    print(len(y_pred))
    y_true = labels[:mini_batch].astype(int)
    
    

    loss = log_loss(y_true, y_pred, labels=[0, 1])
    
    print(y_true, y_pred)
    print("Loss")
    print(loss)
    

    return loss

def train_classifier():
  images=np.load(data_path+'/images.npy')
  labels=np.load(data_path+'/labels.npy')

  N_images = len(images)    
  N_QUBITS = 4
  shots = 1024
  mini_batch = 25

  vqc = ansatz(N_QUBITS)
  np.random.seed(42)
  initial_point = algorithm_globals.random.random(vqc.num_parameters)

  #test part 1
  global pca, AMPLITUDE_ENCODING_N_QUBITS,IMAGE_SIZE, dataset

  AMPLITUDE_ENCODING_N_QUBITS = 4
  IMAGE_SIZE = 28
  N_IMAGES = len(images)

  pca = PCA(n_components=2**AMPLITUDE_ENCODING_N_QUBITS)
  
  data = images.reshape(N_IMAGES, IMAGE_SIZE * IMAGE_SIZE)
  pca.fit(data)
  
  # Apply dimensionality reduction on your data
  images_pca = pca.transform(data)
  # dataset pre-processing
  X_train = []
  dataset = []
  for i in tqdm.tqdm(range(len(images))):
      image_pca = images_pca[i]
      image_pca_min_ = image_pca.min()
      image_pca_positive = image_pca - image_pca_min_
      
      dataset_i = {}
      
      normalized_state, normalization_factor = normalize(image_pca_positive)
      
      dataset_i["image"] = images[i]
      dataset_i["image_vector"] = normalized_state
      dataset_i["label"] = labels[i]
      dataset_i["normalization_factor"] = normalization_factor
      dataset_i["pca_min_"] = image_pca_min_
      X_train.append(normalized_state)
      dataset.append(dataset_i)

  X_train = np.array(X_train)

  #Minimization of the objective_fucntion by a COBYLA method
  minimum = minimize(objective_function, 
                    initial_point, 
                    args=(X_train, labels, shots, mini_batch, N_QUBITS), 
                    method='COBYLA', 
                    tol=1e-6,
                    options={
                        "maxiter": 100
                    }
                    )
  params = minimum.x #Get the solution array
  #Show the cost of the solution array
  #print("Cost: ", objective_function(params))
  print(params)

  return params, minimum

params, minimum = train_classifier()

100%|██████████| 25/25 [00:05<00:00,  4.58it/s]


(2000,)
25
[1 1 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0] [0.44519858402025986, 0.5085602502018445, 0.40632671451209795, 0.20092020244151756, 0.3977107001676153, 0.41472526492673534, 0.6251959425415752, 0.29942951568515996, 0.3723311667474298, 0.6656642171294723, 0.30642829538372324, 0.628821208180102, 0.40582039822943494, 0.6455649988743064, 0.39777949169730054, 0.3522800502215791, 0.6681438022149653, 0.27692736072648644, 0.7358593160250066, 0.4871031485313555, 0.7418107528346649, 0.7457709858003171, 0.40421499000484673, 0.6774790218773664, 0.7173770307390636]
Loss
0.6950484021617176


100%|██████████| 25/25 [00:04<00:00,  5.43it/s]


(2000,)
25
[0 1 0 1 0 0 0 1 0 1 1 1 0 1 0 0 1 0 0 0 0 0 0 1 0] [0.6713651596247816, 0.32797071264017913, 0.6159082805404793, 0.6948126275844074, 0.5181274856264813, 0.4416224649531305, 0.7651560098836326, 0.3891186175970222, 0.8428280495170285, 0.38185086463612594, 0.34402203331214204, 0.39887725924693174, 0.5719200121674068, 0.6428755510693174, 0.7002503570243181, 0.342133916074148, 0.41743510840906595, 0.6569694358548138, 0.7230374549782524, 0.5044971611725263, 0.9487756613093175, 0.4196608441677173, 0.38158536866452547, 0.5494787663284962, 0.81562557728394]
Loss
1.007307867649984


100%|██████████| 25/25 [00:04<00:00,  5.22it/s]


...

(2000,)
25
[0 0 1 0 1 0 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 0 0 0] [0.5064117276658848, 0.582606394530645, 0.43159801524815045, 0.2830759500562336, 0.45220109310645795, 0.37717952523497517, 0.355352641852072, 0.356804406701175, 0.2595683105616583, 0.4265578514056183, 0.34368333191994493, 0.5670550425654248, 0.5031492997027959, 0.39130293939839844, 0.6471133768901106, 0.49704317725029523, 0.2775908288929886, 0.1857201606178082, 0.489306308204554, 0.3399390343856141, 0.44858304782373737, 0.20556068386896048, 0.5829410813160572, 0.3080436388833854, 0.22895089300015303]
Loss
0.5743672965309764


100%|██████████| 25/25 [00:04<00:00,  5.44it/s]


(2000,)
25
[0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0] [0.44337416390285245, 0.22347025276139912, 0.4072912541043265, 0.37766311460352586, 0.3295190573725797, 0.3408275648833128, 0.4871100069392723, 0.2726044317927426, 0.3841790962794557, 0.3774391639455364, 0.3717301345404526, 0.5724713212872472, 0.2838573958376767, 0.32353255401048003, 0.25107948278654957, 0.3507348520007304, 0.16193560909690166, 0.5661635677831014, 0.4054158502441977, 0.24700077035303386, 0.3791385306708585, 0.25520862678452616, 0.507448457872475, 0.3922288476864353, 0.2692315495095034]
Loss
0.532458449969321


100%|██████████| 25/25 [00:04<00:00,  5.29it/s]

(2000,)
25
[0 0 0 1 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0] [0.3417753386555086, 0.5536320468722367, 0.42613222915669113, 0.4412413804602271, 0.3205896641985686, 0.42071812688023735, 0.5023080336617323, 0.4804877689496918, 0.37041189940560204, 0.5967165922866011, 0.468403262165466, 0.3624440501946382, 0.35831932881784306, 0.4905533743320733, 0.3694065635938489, 0.45968188883127525, 0.46445083378450114, 0.28430256309503754, 0.23795509623995448, 0.22151486007673432, 0.2759404667112512, 0.22965115146463683, 0.7141930492003385, 0.4768950328074967, 0.2394397339014594]
Loss
0.5776595803693259
[ 0.55562182  1.14319133  1.0737959   0.40937078  0.0165973   0.50865298
  0.30024442  0.1364341  -0.06497169  0.57702616  0.13664098  1.24139061
  0.45566212  0.64294409  0.57014194  0.03931625  0.87284308  0.49789603
  0.79267549  0.03023329  0.44871798  1.52403189  0.8777964   0.5086433 ]


In [20]:
best_params = params

In [25]:
vqc.

»
q_0: »
     »
q_1: »
     »
q_2: »
     »
q_3: »
     »
«     ┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                                                                                       ├
«     │                                                                                                                                                        │
«q_1: ┤1                                                                                                                                                       ├
«     │  RealAmplitudes(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5],θ[6],θ[7],θ[8],θ[9],θ[10],θ[11],θ[12],θ[13],θ[14],θ[15],θ[16],θ[17],θ[18],θ[19],θ[20],θ[21],θ[22],θ[23]) │
«q_2: ┤2                                                                                                                                                       ├
«     │                                                                                                                                                        │
«q_3: ┤3                                                                                                                                                       ├
«     └────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [27]:
from qiskit import QuantumCircuit
N_QUBITS = 4
vqc = ansatz(N_QUBITS)

vqc = vqc.assign_parameters(best_params)

vqc.qasm(filename="quantum_classifier.qasm")

'OPENQASM 2.0;\ninclude "qelib1.inc";\ngate gate_RealAmplitudes(param0,param1,param2,param3,param4,param5,param6,param7,param8,param9,param10,param11,param12,param13,param14,param15,param16,param17,param18,param19,param20,param21,param22,param23) q0,q1,q2,q3 { ry(0.555621819228932) q0; ry(1.14319132801287) q1; ry(1.07379590145389) q2; ry(0.409370781763752) q3; cx q2,q3; cx q1,q2; cx q0,q1; ry(0.0165973010020706) q0; ry(0.508652978720279) q1; ry(0.300244419920286) q2; ry(0.136434104028988) q3; cx q2,q3; cx q1,q2; cx q0,q1; ry(-0.0649716942665253) q0; ry(0.577026156276957) q1; ry(0.13664098460932) q2; ry(1.24139060584453) q3; cx q2,q3; cx q1,q2; cx q0,q1; ry(0.455662122094092) q0; ry(0.642944089110158) q1; ry(0.570141938819558) q2; ry(0.0393162465417179) q3; cx q2,q3; cx q1,q2; cx q0,q1; ry(0.872843078509701) q0; ry(0.497896034397818) q1; ry(0.792675492672247) q2; ry(0.0302332873841969) q3; cx q2,q3; cx q1,q2; cx q0,q1; ry(0.448717981940666) q0; ry(1.52403189005694) q1; ry(0.877796397476

In [28]:
!ls

data  quantum_classifier.qasm  sample_data


In [30]:
from google.colab import files

files.download('quantum_classifier.qasm')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
vqc = ansatz(N_QUBITS)

In [17]:
params

NameError: ignored